In [2]:
import pyodbc
import sys

class Restaurant():

    def __init__(self):
        self.totalamt = 0
        self.order = []   #stores the ordered items in list
        self.conn = None
        self.cursor = None

    def Connection(self):
        try:
            self.conn = pyodbc.connect(
                'Driver={SQL Server};'
                'Server=LAPTOP-OB6U7IV1\\SQLEXPRESS;'
                'Database=HOTEL_PYDAY9;'
                'Trusted_Connection=yes;'
            )

            self.cursor = self.conn.cursor()
            print("Connected Successfully\n")

        except Exception:
            print("Database Connection Error:", sys.exc_info())

    def Menu(self):
        try:
            print("---------------------- MENU ----------------------")
            self.cursor.execute("SELECT MenuID, Item, Price, Category FROM Menu")

            records = self.cursor.fetchall()

            print("{:<8} {:<20} {:<10} {:<15}".format("ID", "Item", "Price", "Category"))
            print("-" * 55)

            for row in records:
                print("{:<8} {:<20} {:<10} {:<15}".format(row[0], row[1], row[2], row[3]))

        except Exception:
            print("Error fetching menu:", sys.exc_info())

    def TakeOrder(self):

        while True:
            try:
                menu_id = int(input("\nEnter MenuID to order: "))

                self.cursor.execute("SELECT Item, Price FROM Menu WHERE MenuID = ?", (menu_id,))
                row = self.cursor.fetchone()

                if row:
                    item_name = row[0]
                    price = float(row[1])

                    qty = int(input("Enter Quantity: "))
                    total = price * qty

                    
                    self.cursor.execute(
                        "INSERT INTO Orders (MenuID, Qty, TotalAmt) VALUES (?, ?, ?)",
                        (menu_id, qty,total)
                    )

                    self.conn.commit()

                    self.totalamt += total
                    self.order.append((item_name, qty, total))

                    print(f"{qty} {item_name}(s) added successfully")

                else:
                    print("Invalid MenuID")

                more = input("Add more items? (yes/no): ").lower()
                if more not in ("yes", "y"):
                    break

            except ValueError:
                print("Invalid Input! Please enter numbers properly.")
            except Exception:
                print("Error while placing order:", sys.exc_info()[1])

    def DeliverOrder(self):
        if not self.order:
            print("No Order Placed")
        else:
            print("\nYour order is being delivered.")

    def PrintBill(self):

        if not self.order:
            print("No Bill Available")
            return

        choice = input("Do you want Bill? (yes/no): ").lower()

        if choice in ("yes", "y"):

            print("\n--------------------------------------------")
            print("{:^45}".format("HOTEL SIMPLY SOUTH"))
            print("------------------- BILL -------------------")

            print("{:<20} {:>8} {:>10}".format("Item Name", "Qty", "Total"))
            print("-" * 45)

            for item in self.order:
                print("{:<20} {:>8} {:>10}".format(item[0], item[1], item[2]))

            print("-" * 45)
            print("{:<20} {:>18}".format("Grand Total:", self.totalamt))
            print("--------- THANK YOU VISIT AGAIN !! ---------")
            print("--------------------------------------------")

        else:
            print("Thank You for Visiting!")

    def CloseConnection(self):
        if self.cursor:
            self.cursor.close()
        if self.conn:
            self.conn.close()
            print("Connection Closed")


Rest_obj = Restaurant()

Rest_obj.Connection()
Rest_obj.Menu()
Rest_obj.TakeOrder()
Rest_obj.DeliverOrder()
Rest_obj.PrintBill()
Rest_obj.CloseConnection()


Connected Successfully

---------------------- MENU ----------------------
ID       Item                 Price      Category       
-------------------------------------------------------
1        Poha                 30.00      Breakfast      
2        Upma                 35.00      Breakfast      
3        Sheera               40.00      Breakfast      
4        Paneer Makhanwala    200.00     Meal           
5        Kaju Curry           300.00     Meal           
6        Chicken Curry        280.00     Meal           
7        Dosa                 50.00      Breakfast      
8        Uttapa               70.00      Meal           
9        Dal Tadka            180.00     Meal           
10       Dal Khichdi          100.00     Meal           
11       Roti                 50.00      Meal           
12       Chapati              40.00      Meal           



Enter MenuID to order:  4
Enter Quantity:  1


1 Paneer Makhanwala(s) added successfully


Add more items? (yes/no):  yes

Enter MenuID to order:  11
Enter Quantity:  6


6 Roti(s) added successfully


Add more items? (yes/no):  no



Your order is being delivered.


Do you want Bill? (yes/no):  yes



--------------------------------------------
             HOTEL SIMPLY SOUTH              
------------------- BILL -------------------
Item Name                 Qty      Total
---------------------------------------------
Paneer Makhanwala           1      200.0
Roti                        6      300.0
---------------------------------------------
Grand Total:                      500.0
--------- THANK YOU VISIT AGAIN !! ---------
--------------------------------------------
Connection Closed
